In [ ]:
!python -m spacy download en_vectors_web_lg

In [ ]:
import pandas as pd

df = pd.read_csv('LOCAL_PATH_TO_DATASET')
df = df[['Emotion','Statement']]
display(df.head())

In [ ]:
from sklearn.utils import class_weight
import numpy as np

class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(df['Emotion'])
                                               ,df['Emotion'])
y = df.pop('Emotion')
X = df

In [ ]:
def process_text(document):
     
    # Remove extra white space from text
    document = re.sub(r'\s+', ' ', document, flags=re.I)
         
    # Remove all the special characters from text
    document = re.sub(r'\W', ' ', str(document))
 
    # Remove all single characters from text
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
 
    # Converting to Lowercase
    document = document.lower()
 
    # Word tokenization       
    tokens = document.split()
      
    lemma_txt = [stemmer.lemmatize(word) for word in tokens]
    # Remove Drop words 
    lemma_no_stop_txt = [word for word in lemma_txt if word not in en_stop]
        
    tokens = [word for word in tokens if len(word) > 3]
                 
    clean_txt = ' '.join(lemma_no_stop_txt)
 
    return clean_txt

In [ ]:
from tqdm import tqdm
import numpy as np
import nltk
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
import re
from nltk import WordNetLemmatizer
stemmer = WordNetLemmatizer()

nltk.download('wordnet')
nltk.download('stopwords')  

df['preprocessedStatement'] = df.Statement.apply(process_text)

In [ ]:
clean_corpus = df.preprocessedStatement.tolist()
clean_corpus[:5]

In [ ]:
import en_vectors_web_lg
nlp = en_vectors_web_lg.load()  

In [ ]:
document = nlp.pipe(token for token in clean_corpus)
vectors = np.array([word_tokens.vector for word_tokens in document])
print(vectors.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X_train,X_test, y_train, y_test = train_test_split(vectors,y, stratify=y, test_size=0.05, random_state=0)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model_lr = LogisticRegression(multi_class='ovr', max_iter=1000) #Insert class_weight = class_weight if training on Meld-dd dataset
model_lr.fit(X_train, y_train)

y_pred_lr = model_lr.predict(X_test)

print(classification_report(y_test, y_pred_lr))

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

model_lr2 = LogisticRegression(max_iter=1000) #Insert class_weight = class_weight if training on Meld-dd dataset

model_lr2 = OneVsRestClassifier(model_lr2)

model_lr2.fit(X_train, y_train)

y_pred_lr2 = model_lr2.predict(X_test)

print(classification_report(y_test, y_pred_lr2))

In [ ]:
from sklearn.svm import SVC

model_svm = SVC(decision_function_shape='ovo') #insert class_weight = 'balanced' if training on Meld-dd dataset
model_svm.fit(X_train, y_train)

y_pred_svm = model_svm.predict(X_test)

print(classification_report(y_test, y_pred_svm))

In [ ]:
from sklearn.multiclass import OneVsOneClassifier

model_svm2 = SVC() #insert class_weight = 'balanced' if training on Meld-dd dataset

model_svm2 = OneVsOneClassifier(model_svm2)

model_svm2.fit(X_train, y_train)
  
y_pred_svm2 = model_svm2.predict(X_test) 

print(classification_report(y_test, y_pred_svm2))

In [ ]:
list(encoder.inverse_transform([0,1,2,3,4,5,6]))